In [1]:
import sys, os, time, datetime    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.actions.mouse_button import MouseButton
from selenium.webdriver.common.by import By

In [2]:
#크롬 웹브라우저 실행
driver = webdriver.Chrome()
url = "https://finance.naver.com/sise/theme.nhn?&page=1"
driver.get(url)
time.sleep(2)

In [15]:
# 7페이지 for문으로 반복
for i in range(7):
    html = requests.get('https://finance.naver.com/sise/theme.nhn?&page={}'.format(i + 1))
    table = pd.read_html(html.text)
    if i == 0:
        stock = table[0].dropna().reset_index(drop = True)
    else:
        stock1 = table[0].dropna().reset_index(drop = True)
        stock = pd.concat([stock, stock1])

stock = stock.reset_index(drop = True)
stock.drop(stock.columns[1:], axis=1, inplace=True) ### 첫째열 제외하고 삭제
stock = stock.set_axis(['theme'], axis=1) ### 칼럼 이름 변경
pd.options.display.max_rows = 4000
print(stock)

                      theme
0              야놀자(Yanolja)
1                   인터넷 대표주
2                   일자리(취업)
3              가상화폐(비트코인 등)
4                    건설 대표주
5            항공/저가 항공사(LCC)
6                       홈쇼핑
7                        은행
8                        게임
9                      종합상사
10                       통신
11           STO(증권형 토큰 발행)
12                     손해보험
13           SNS(소셜네트워크서비스)
14              K-뉴딜지수(인터넷)
15              NFT(대체불가토큰)
16                    음원/음반
17                       증권
18              모바일게임(스마트폰)
19        카카오뱅크(kakao BANK)
20                      창투사
21              반도체 대표주(생산)
22               K-뉴딜지수(게임)
23             케이블TV SO/MSO
24                     애플페이
25                SI(시스템통합)
26                      아이폰
27                  환율하락 수혜
28                      지주사
29                     공작기계
30             모더나(MODERNA)
31                     테마파크
32            UAM(도심항공모빌리티)
33                리츠(REITs)
34              두나무(

In [10]:
# 2차전지 테마의 인덱스 찾기
# l = []
# for i, s in stock.iterrows():
#     t = s.to_string()
#     j = t.find('2차전지')
#     if j > 0:
#       l.append([t[9:], i])  
# print(l)

# 총 279개 테마, 각 페이지별로 40개씩, 40으로 나누고 나머지로 행위치
# 그런데 페이지를 넘길때마다 어떤 테마는 2개씩 나오는 에러가로 행위치를 특정해서 클릭하는 방식은 안되고
# 모든 테마를 들어가서 테마이름별 No 정보를 가지고 와야겠음

[['2차전지(소재/부품)', 125], ['2차전지', 181], ['2차전지(전고체)', 213], ['2차전지(장비)', 234], ['K-뉴딜지수(2차전지)', 236], ['2차전지(LFP/리튬인산철)', 251], ['2차전지(생산)', 270]]


In [13]:
# l_location=[]
# for idx, i in enumerate(l):
#     j, k = divmod(i[1], 40)
#     l_location.append([i[0],j+1,k])
# print(l_location)

[['2차전지(소재/부품)', 4, 5], ['2차전지', 5, 21], ['2차전지(전고체)', 6, 13], ['2차전지(장비)', 6, 34], ['K-뉴딜지수(2차전지)', 6, 36], ['2차전지(LFP/리튬인산철)', 7, 11], ['2차전지(생산)', 7, 30]]


In [38]:
### 1페이지부터 7페이지까지 모두 이동하면서 모든 행을 클릭해서 테마이름별 No 정보 취함
def theme_url_load(page):
    url = ('https://finance.naver.com/sise/theme.nhn?&page={}'.format(page))
    driver.get(url)
    time.sleep(1)
k=1
l_data = []

first_url = "https://finance.naver.com/sise/theme.naver"
driver.get(first_url)
print('테마개수 : {0}'.format(len(stock)))

for i in range(1, 7):
    theme_url_load(i)
    for j in range(1,40):
        if k <= len(stock):
            print(i, j, k)
            quotient, remainder = divmod(j, 5)
            if remainder == 0:
                quotient-=1
            table_row = j + (quotient+1) * 3    
                        
            xpath = ('//*[@id="contentarea_left"]/table[1]/tbody/tr[{}]/td[1]/a'.format(table_row))
            print(table_row, xpath)
            
            clickable = driver.find_element(By.XPATH, xpath)
            ActionChains(driver)\
                .click(clickable)\
                .perform()
            
            url = driver.current_url ### 페이지의 URL을 받음           
            reqs = requests.get(url)
            # using the BeautifulSoup module
            soup = BeautifulSoup(reqs.text, 'html.parser')

            for title in soup.find_all('title'):
                str_title = title.get_text().strip()
            x = str_title.find(':')
            str_title = str_title[:x-1]
            print(str_title, url)
            l_data.append([str_title, url])
            k += 1
            driver.back()
            time.sleep(1)

print(l_data)  

테마개수 : 279
1 1 1
4 //*[@id="contentarea_left"]/table[1]/tbody/tr[4]/td[1]/a
야놀자(Yanolja) https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=483
1 2 2
5 //*[@id="contentarea_left"]/table[1]/tbody/tr[5]/td[1]/a
인터넷 대표주 https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=49
1 3 3
6 //*[@id="contentarea_left"]/table[1]/tbody/tr[6]/td[1]/a
일자리(취업) https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=324
1 4 4
7 //*[@id="contentarea_left"]/table[1]/tbody/tr[7]/td[1]/a
가상화폐(비트코인 등) https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=331
1 5 5
8 //*[@id="contentarea_left"]/table[1]/tbody/tr[8]/td[1]/a
건설 대표주 https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=154
1 6 6
12 //*[@id="contentarea_left"]/table[1]/tbody/tr[12]/td[1]/a
항공/저가 항공사(LCC) https://finance.naver.com/sise/sise_group_detail.naver?type=theme&no=166
1 7 7
13 //*[@id="contentarea_left"]/table[1]/tbody/tr[13]/td[1]/a
홈쇼핑 https://finance.naver.com

In [ ]:
path = 'C:/Users/Admin/py_stock_analysis/'

df = pd.DataFrame(l_data)
df.to_csv(path+'thema_url.csv', index=False)